In [1]:
import numpy as np
import solar_sensor_env
import itertools
import functools

In [2]:
solarfname = 'training_12'
solarrecord = solar_sensor_env.get_generated_power(solarfname)
monthrecord = solar_sensor_env.get_month(solarrecord,8)
deltat = 3
num_ts = int(24/deltat)
powerseries = solar_sensor_env.downsample(monthrecord, factor=int(48/num_ts))
#sanity check
print(sum(powerseries[0:8])/8, sum(monthrecord[0:48])/48)

0.4293984145236512 0.4293984145236512


In [3]:
def generate_all_dutycycle_plans(times = 8):
    possibilities = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    all_combs = itertools.product(possibilities, repeat=times)
    for comb in all_combs:
        yield comb
def calculate_consumed(duty_cycle):
    discharge_voltage = 3.7
    deltat = 3
    on_power = 56+45+15#mAh pyboard plus digimesh plus accel
    off_power = 45#mAh
    deepsleeppower = 0.5
    if duty_cycle >0:#sleeping
        used_power = (on_power*discharge_voltage*deltat*duty_cycle)+(off_power*discharge_voltage*deltat*(1-duty_cycle))
    else:#either pre-sleep or awake
        used_power = (deepsleeppower*deltat*discharge_voltage)
    return used_power
def evaluate_cost(harvest_prediction,aplan, deltat = 3):
    total_consumed = sum([calculate_consumed(duty) for duty in aplan]) #in MWh
    total_harvested = sum(harvest_prediction)*1000
    return np.abs(total_consumed - total_harvested)
    

min all i in day 
(duty_i*on_cost +(1-duty_i)*off_cost...(1-duty_n)*off_cost) - harvested
s.t.
0<duty_i<1

maximise the sum of duty cycle,
constrain such that the battery ending must be greater eq than the battery starting,
and no battery is less than zero

In [4]:
cost_evaluator = functools.partial(evaluate_cost, powerseries[0:8])

In [5]:
plans= generate_all_dutycycle_plans()
min_cost_plan = min(((plan,cost_evaluator(plan)) for plan in plans), key=lambda i:i[1])

In [6]:
min_cost_plan

((0, 0, 0, 0, 0.1, 0.3, 1, 0.4), 3.5926838107902768)